# Geoapify Geocode Demo

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy
# Import the API key
api_key = "60a6ffdd990b65ca16853197a6e0c84c"

In [27]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 600


In [63]:
# Test to see the json data
lat = 40.7608
lon = 111.8910
start = 1606460400
end = 1606546800
historical_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={api_key}"
test_call = requests.get(historical_url).json()
print(test_call['list'][0]['main'])

{'aqi': 2}


In [ ]:
historical_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?"

city_data = []

start = 1606460400
end = 1701068400
# Loop through all the cities in our list to fetch weather data
for i, (lat, lon) in enumerate(lat_lngs):

    # Create endpoint URL with each city
    city_url = f'{historical_url}lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}'

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_air = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_coords =  city_url['coord']
        city_SO2 = city_url['list'][0]['components']['so2']
        city_NO2 = city_url['list'][0]['components']['no']
        city_PM10 = city_url['list'][0]['components']['pm10']
        city_PM25 = city_url['list'][0]['components']['pm2_5']
        city_O3 = city_url['list'][0]['components']['o3']
        city_CO2 = city_url['list'][0]['components']['co']
        city_aqi = city_url['list'][0]['main']

        # Append the City information into city_data list
        city_data.append({"Coordinates": city_coords, 
                          "Sulphur dioxide (SO2)": city_SO2, 
                          "Nitrogen dioxide (NO2)": city_NO2, 
                          "Particulates 2.5": city_PM25,
                          "Particulates 10": city_PM10,
                          "Ozone (O3)": city_O3,
                          "Carbon monoxide (CO)": city_CO2,
                          "Air Quality Index": city_aqi
                        })

    # If an error is experienced, skip the city
    except:
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
pollution_df = pd.DataFrame(city_data)
pollution_df.head()